In [1]:
import os
import numpy as np
import time
import pickle

In [2]:
def smooth_landmark(curr_landmark, prev_landmark, next_landmark):
    landmarks = [curr_landmark, prev_landmark, next_landmark]
    s_landmarks = []
    
    for l in landmarks:
        if l is not None:
            s_landmarks.append(l)
            
    if len(s_landmarks) == 0:
        return None
    elif len(s_landmarks) == 1:
        return s_landmarks[0]
    elif len(s_landmarks) == 2:
        return (s_landmarks[0] + s_landmarks[1]) / 2
    elif len(s_landmarks) == 3:
        return (s_landmarks[0] + s_landmarks[1] + s_landmarks[2]) / 3

In [3]:
def get_face_moves(mid_landmarks):
    face_moves = []
#     the first move is zero
    face_moves.append(np.array([0,0]))
    for i in range(len(mid_landmarks)-1):
        move = mid_landmarks[i+1] - mid_landmarks[i]
        face_moves.append(move)
    return face_moves

In [4]:
def get_mid_landmark(landmarks):
    mid_landmarks = []
    for i in range(len(landmarks)):
        curr_landmark = prev_landmark = next_landmark = None
        if i == 0:            
            if landmarks[i] is not None:
                curr_landmark = landmarks[i]
            if landmarks[i+1] is not None:
                next_landmark = landmarks[i+1]
        elif i == (len(landmarks)-1):            
            if landmarks[i] is not None:
                curr_landmark = landmarks[i]
            if landmarks[i-1] is not None:
                prev_landmark = landmarks[i-1] 
        else:
            if landmarks[i] is not None:
                curr_landmark = landmarks[i]
            if landmarks[i-1] is not None:
                prev_landmark = landmarks[i-1] 
            if landmarks[i+1] is not None:
                next_landmark = landmarks[i+1]
                
        landmark_s = smooth_landmark(curr_landmark, prev_landmark, next_landmark)
        mid_landmark = [0, 0]
        if landmark_s is not None:
            mid_landmark[0] = landmark_s[0,:].mean()
            mid_landmark[1] = landmark_s[1,:].mean()
        mid_landmarks.append(np.array(mid_landmark))
    return mid_landmarks

In [5]:
annotation_path = '/nas/database/Big5/gt/annotation_training.pkl'
input_dir = '/nas/project_data/B1_Behavior/hakim/FI/3DDFA/train'
output_dir = '/nas/project_data/B1_Behavior/hakim/FI/pose_face_moves/train'

with open(annotation_path, 'rb') as f:
    annotation = pickle.load(f, encoding='latin1')
videos = list(annotation['extraversion'].keys())

for i, video in enumerate(videos):
    video_name = video.rsplit(".")[0] + "." + video.rsplit(".")[1]
    input_name = video_name + '.pickle'
    output_name = video_name + '.npy'
    input_path = os.path.join(input_dir, input_name)
    output_path = os.path.join(output_dir, output_name)
    print(f'Processing {i} {video_name}')
    
    with open(input_path, 'rb') as f:
        tddfa_output = pickle.load(f, encoding='latin1')
    
    landmarks = []
    for item in tddfa_output:
        if item is None:
            landmarks.append(None)
        else:
            landmarks.append(item['landmark'])
    
    mid_landmarks = get_mid_landmark(landmarks)
    face_moves = get_face_moves(mid_landmarks)
    features = []
    
    for i in range(len(tddfa_output)):
        if tddfa_output[i] is None:
            feature = [0, 0, 0] + list(face_moves[i])
        else:
            feature = tddfa_output[i]['pose'] + list(face_moves[i])
        features.append(feature)   
        
    features = np.array(features)
    
    with open(output_path, 'wb') as f:
        np.save(f, features)

Processing 0 J4GQm9j0JZ0.003
Processing 1 zEyRyTnIw5I.005
Processing 2 nskJh7v6v1U.004
Processing 3 6wHQsN5g2RM.000
Processing 4 dQOeQYWIgm8.000
Processing 5 eHcRre1YsNA.000
Processing 6 vZpneJlniAE.005
Processing 7 oANKg9_grdA.004
Processing 8 VuadgOz6T7s.000
Processing 9 7nhJXn9PI0I.001
Processing 10 tEQEKN07KgQ.001
Processing 11 I5x9T9ftW18.005
Processing 12 dh6iOU2RtTA.003
Processing 13 gsleSGEZHAs.004
Processing 14 vhugKRUnd-c.001
Processing 15 kn6I8LdQFN0.004
Processing 16 M5_x5J-H2I0.003
Processing 17 w_ExyXsnw2A.000
Processing 18 DnTtbAR_Qyw.004
Processing 19 rIHcq2E67Nc.005
Processing 20 De4i7-FX9Og.004
Processing 21 cA43Gfcg0QA.000
Processing 22 pZxqWp0e-Ik.000
Processing 23 D3AomvNhR1k.003
Processing 24 Fi1ILrwQpSY.003
Processing 25 Xfmu-7JuDGg.002
Processing 26 YzNbGMNr3z0.001
Processing 27 aFVFvJNnFt0.000
Processing 28 cgT-3CHBmKs.000
Processing 29 P_wDFLJnW-o.004
Processing 30 t0_DAgeU4nM.002
Processing 31 TTuysItuSdo.004
Processing 32 nUbhRInpVbA.004
Processing 33 kL-Cea

Processing 272 53gtUkC7IZM.001
Processing 273 L5PkZA1ErYY.002
Processing 274 eZwGkhUa0TQ.005
Processing 275 Wby2oMliGz0.005
Processing 276 PxkMDzORBPE.005
Processing 277 WUNXs4r_xCU.003
Processing 278 sBHv1jYmZQE.004
Processing 279 me15sHPOjKo.005
Processing 280 LBrmHHzyIiY.002
Processing 281 uqd_cEJMJ4o.003
Processing 282 oqXx6ipHfUE.000
Processing 283 i4IVTm3uvoM.000
Processing 284 oU7zAWm_01o.003
Processing 285 cOirMOCpdo8.000
Processing 286 Ft0ahViXZPc.003
Processing 287 bcRPLKygrNk.000
Processing 288 OLFhKCgexRU.001
Processing 289 ic_LECbCm5M.004
Processing 290 Shf0JB_MutI.001
Processing 291 GCkOL2jLSQA.005
Processing 292 51KRxB3g7A8.000
Processing 293 USicjVA1trA.003
Processing 294 YiX6thczM1w.002
Processing 295 HrddpXxWFns.002
Processing 296 voXr6-ykd1w.000
Processing 297 3LAaFUSGvsU.001
Processing 298 XvV4YdhpC2w.001
Processing 299 K-iwJMxjyx4.000
Processing 300 kL-CeaXG9jM.002
Processing 301 OmXuhT4yJpo.000
Processing 302 ROTx-0q1F9M.001
Processing 303 vR7YF7Zv5-Q.000
Processi

Processing 537 tiVKJWLzmk4.001
Processing 538 jmbAPNTJ3Vk.002
Processing 539 EiXQ9RErt6c.002
Processing 540 UKHSKkb0V1o.005
Processing 541 hG0I7DLSSYg.002
Processing 542 m7zdWVDT-34.004
Processing 543 kzDQXSGGomg.002
Processing 544 ztyBhnjtrz0.001
Processing 545 3Q3SB1dVAng.000
Processing 546 -VTqcHNgH7M.004
Processing 547 M39PNQ8Omhc.005
Processing 548 EcOC7ZVRSg4.001
Processing 549 OxaBanUE6KE.002
Processing 550 vYVqhg_VGP8.000
Processing 551 T9xSEBSaL7Q.003
Processing 552 qkmEt0ofGq8.002
Processing 553 cWZudzeJVXg.000
Processing 554 TaL5JleXDP8.005
Processing 555 MbufNmC0JL0.004
Processing 556 Bk9Hxn7Drxw.003
Processing 557 8XJCdulCPkM.004
Processing 558 LtBYojIGYEc.001
Processing 559 djsoQkroHpo.001
Processing 560 TIRrlL71dEQ.000
Processing 561 1hpZ2ecWqtI.005
Processing 562 Os7L5C-i9Fw.003
Processing 563 bt-ev53zZWE.001
Processing 564 A6ZWTfKPLPE.003
Processing 565 e3NaBdG-ZmM.000
Processing 566 mZ3vypXAfHg.000
Processing 567 zF46tAI2PQc.004
Processing 568 T1tJTwlGqOM.000
Processi

Processing 802 A3StIKMjn4k.001
Processing 803 MnARi5OWW9Q.000
Processing 804 cOirMOCpdo8.005
Processing 805 uTluOTPA7e4.001
Processing 806 mhF4kYTlVUE.004
Processing 807 E6OBPHKkm6U.002
Processing 808 VugKq4gOi50.005
Processing 809 rqfvIjzegpI.005
Processing 810 JO44XCaQGVY.003
Processing 811 KYvbRwTCtaU.002
Processing 812 0G9vplL8ae8.001
Processing 813 lLKwlPLdYig.000
Processing 814 CdY1ZLrWTtg.000
Processing 815 o_kTWEEjtyw.003
Processing 816 uHrP02iwW1o.002
Processing 817 w989xx44UQI.004
Processing 818 GS7cBLgZKgk.000
Processing 819 3wgZrRMosvY.003
Processing 820 kDJigDGDJTo.000
Processing 821 r1v6EmZv7-o.002
Processing 822 L_gmlaz-0s4.001
Processing 823 ioaBnmfoHgo.000
Processing 824 ODDGS0hTcUc.001
Processing 825 voSLxKtKr0Q.005
Processing 826 cOossY5pTrU.002
Processing 827 2oiXF5_gTyk.003
Processing 828 DyZlhseSuec.002
Processing 829 NoHext7JXYU.000
Processing 830 4yogPbHFQ9o.005
Processing 831 THO-md3eEkA.003
Processing 832 vP6TUda99tc.005
Processing 833 gAWA-fcIa_M.005
Processi

Processing 1069 x6Tluj4dJTU.002
Processing 1070 NOrjfF0-pQ4.004
Processing 1071 B-OJ_xZILLg.001
Processing 1072 JFKZEGDhcRs.005
Processing 1073 MajYvFTkKnk.004
Processing 1074 K7vBdKrcFKo.001
Processing 1075 7nJPNkWZDOY.003
Processing 1076 J0JrzUmZGcA.002
Processing 1077 B3zFJhrmqwg.002
Processing 1078 g0Zlp-F6Z6E.001
Processing 1079 0mym1CooiTE.001
Processing 1080 UGp1GNK0GWg.000
Processing 1081 F4UeAogUMMs.005
Processing 1082 19cplBTXyoY.002
Processing 1083 gq0yaEv6gSE.000
Processing 1084 3y3au0hVfm8.001
Processing 1085 vrMlwwTLWIE.005
Processing 1086 mnRuiwFx_Z8.005
Processing 1087 zxuPCshTSOs.002
Processing 1088 yUESJPS0wS8.004
Processing 1089 Y1LrgyCJtpU.003
Processing 1090 ADa9rkwF2uA.002
Processing 1091 Lq4uvcHNiy8.003
Processing 1092 dLaqub4BEsA.003
Processing 1093 wzCdM6Fe_4A.001
Processing 1094 ZX3okNzASCs.000
Processing 1095 c672aEuUa-g.002
Processing 1096 cFGjJg-2zOk.001
Processing 1097 qW0TYVtzq5A.000
Processing 1098 IHJn3UHXCMs.004
Processing 1099 QipgesEKSNA.004
Processi

...

Processing 4701 DXYRJ8PReoE.002
Processing 4702 P6g1mZ3f76U.003
Processing 4703 bkzgccBFztM.000
Processing 4704 VjYqagJ4xt8.004
Processing 4705 8GFElKeREfQ.005
Processing 4706 ySEeioc339E.003
Processing 4707 RrvG7JlKoXM.002
Processing 4708 bAxlTgobVHE.002
Processing 4709 dkhnv1EBvKI.003
Processing 4710 8py6SksNsZQ.001
Processing 4711 ua5c716lu5s.002
Processing 4712 VAZLQqJa9rQ.004
Processing 4713 p7-JUvStF4w.000
Processing 4714 si_gZCrLa4A.002
Processing 4715 9DF_-PXbpkg.001
Processing 4716 JEhC58vobs4.001
Processing 4717 qoZwA4ESb7g.001
Processing 4718 _LWrd9t4oZw.003
Processing 4719 xyOq5SL0Evc.000
Processing 4720 KGRUaRe3r7I.002
Processing 4721 AWe6ZZUmlZw.000
Processing 4722 U-PRay-woXU.005
Processing 4723 9cHxDnk6SUs.000
Processing 4724 n8IiQJyqjiE.000
Processing 4725 _mtiuHyOFXg.005
Processing 4726 8z-WF2UNRb8.002
Processing 4727 lnawWqnGpMc.002
Processing 4728 _G3kw9HwCqY.004
Processing 4729 8XBprf4NyOg.004
Processing 4730 nEm44UpCKmA.000
Processing 4731 MT3tOBpG5II.000
Processi

Processing 4962 fiVORaERbP0.004
Processing 4963 Y8gv_ZK3w4A.002
Processing 4964 g00oS-Kt248.001
Processing 4965 WuIKjBb7s4Q.003
Processing 4966 NkFJuN8fBDU.001
Processing 4967 VOhsEbDjZ9E.001
Processing 4968 iAgu-wDe2MQ.002
Processing 4969 sqnW6xXYTKg.001
Processing 4970 KmHe8urs3LI.003
Processing 4971 pQy4omHIf7o.003
Processing 4972 lvae9lRUugE.004
Processing 4973 KRo-x2uoHUg.003
Processing 4974 oYiWW-Qc0rs.001
Processing 4975 AFvUxDwJB84.003
Processing 4976 ZXeO5dRFrj0.001
Processing 4977 AS8snUf9eX0.001
Processing 4978 p-OcwNFQB0U.001
Processing 4979 C8xZ0vhrrFE.005
Processing 4980 ROTx-0q1F9M.004
Processing 4981 Jhz5ssCKaug.002
Processing 4982 uMCRfie1lkc.003
Processing 4983 Jog1-pYOSD0.005
Processing 4984 nUazOUcW_AM.004
Processing 4985 TD3H2DOSi1Y.001
Processing 4986 HCkDd0UnUgo.003
Processing 4987 HL-K_PZXFR4.005
Processing 4988 Yj36y7ELRZE.000
Processing 4989 _2DSmy36OEM.001
Processing 4990 yIe7Rp54-bU.004
Processing 4991 X1EjfXX3NTk.004
Processing 4992 nIwFocnmRFY.003
Processi

Processing 5223 8-OKP1nf19E.001
Processing 5224 IL5VgGakSNw.005
Processing 5225 8Z6PrQdqffA.002
Processing 5226 Vy-Lr_iM5qo.001
Processing 5227 1mdMhaq5p4w.004
Processing 5228 iaCYb7um0Lw.000
Processing 5229 xDjhf8wrNWQ.005
Processing 5230 taSAaWKcmTw.000
Processing 5231 LXmwDq89LdA.005
Processing 5232 twO6ceVTuII.002
Processing 5233 S-jhg6Y5OrE.001
Processing 5234 Z_STuUHKQTg.002
Processing 5235 HUHFlCs3YJ8.003
Processing 5236 -Osv5l1vrNU.004
Processing 5237 F0VbEO_0Ybc.002
Processing 5238 -55DRRMTppE.005
Processing 5239 merppcmY6y0.003
Processing 5240 NTSMogqhvBg.002
Processing 5241 Cl7vQ9QR0es.001
Processing 5242 S0QIXcQGG-g.004
Processing 5243 Fi1ILrwQpSY.001
Processing 5244 MZkDsB04kt8.004
Processing 5245 emHDoDcht44.001
Processing 5246 lNY0BdE4NqY.000
Processing 5247 xX5Imr7UmaA.002
Processing 5248 nKx64KH7Xks.001
Processing 5249 lx6A8s_Ega8.000
Processing 5250 PHv6CzBIC5E.005
Processing 5251 NdAJGXf8aos.002
Processing 5252 GWOtluILlk4.005
Processing 5253 XN5vpJU3ifw.000
Processi

Processing 5482 LBrmHHzyIiY.000
Processing 5483 nghDz0el8iI.000
Processing 5484 RWfY3Xk0XPQ.002
Processing 5485 0axZSeaUbfs.004
Processing 5486 _UnoRkHkYB8.003
Processing 5487 Ya9Szm73LfE.002
Processing 5488 8hGOOynn3ZU.003
Processing 5489 LSqijVxP0EM.000
Processing 5490 m04e9ylCoK0.000
Processing 5491 yp-tfq1NxBk.003
Processing 5492 1fsFL_qNYlQ.001
Processing 5493 ZoHY-rWJsyw.000
Processing 5494 12Ezy1y1cWY.005
Processing 5495 BjQ95T-Dw1U.005
Processing 5496 7Y4cUPFEt-4.000
Processing 5497 jZl3Rq1MRQg.003
Processing 5498 2bAsXLQjlt8.002
Processing 5499 OMHlfDF99Mw.000
Processing 5500 iGBVrJMr62U.002
Processing 5501 simc6hmrUOE.005
Processing 5502 _7hGh6VU4IU.003
Processing 5503 _hR9AD-eIy8.005
Processing 5504 8n6G3V3-Tk8.002
Processing 5505 nDcl8Jo2tYg.002
Processing 5506 x0CZuHnJ0Hs.005
Processing 5507 isqmQc9AfBc.001
Processing 5508 O3_j0d7mq7k.002
Processing 5509 USO-o9dHJ3U.000
Processing 5510 h_kjgcJ_bqU.002
Processing 5511 2hhqEWiv4eI.000
Processing 5512 CpmR-91GxmY.002
Processi

Processing 5745 d9_YDJdF-7I.003
Processing 5746 SbtIS7XwkLU.005
Processing 5747 _hR9AD-eIy8.000
Processing 5748 LGgx7RVdL2Y.004
Processing 5749 LBKbubsgpE8.003
Processing 5750 ixPqs7L8oAE.005
Processing 5751 38fE5B4ghw0.000
Processing 5752 uf_sIIw4zxY.005
Processing 5753 RsYSOFRFszk.004
Processing 5754 PKYh4sungwI.003
Processing 5755 8XBprf4NyOg.002
Processing 5756 sRuO_Fz3eog.004
Processing 5757 lkdQjIunxSE.000
Processing 5758 bej-DbxTkWA.000
Processing 5759 J8YpOUmN_aI.003
Processing 5760 3tPq9fNOXZQ.003
Processing 5761 TwWDFpN3occ.003
Processing 5762 MK7KveAeb6Y.002
Processing 5763 dIa0U29hVvU.001
Processing 5764 -DOqN0d8KHw.002
Processing 5765 M0O87_LiyMw.005
Processing 5766 vZPLbzF0aSg.001
Processing 5767 YjXUMygVw7U.005
Processing 5768 jwbHWwySJW8.004
Processing 5769 W2AB3UUyvwI.002
Processing 5770 GquB5W7bzFo.003
Processing 5771 EffEoI1l1zE.000
Processing 5772 Tm44tg1zPwU.004
Processing 5773 6-_1-vBNl_0.004
Processing 5774 3df_Uk9EmwU.005
Processing 5775 9UEBU_yiTPs.002
Processi